In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import sys
import time
from pathlib import Path

try:
    sys.path.append(str(Path().cwd().parent))
except IndexError:
    pass

from cqp.bitrate_model.plot import *
from cqp.bitrate_model.util import *
from cqp.cqp_ml_model_train import *

In [ ]:
MODEL_PATH = "cqp/models/x264-cif.h5"

max_parameter = MaxBitrateParameter()

In [ ]:
data_files = list(Path.cwd().joinpath("notebooks").joinpath("data").glob("*.csv"))

dfw = DataFrameWrapper(read_df_by_keys(data_files, KEYS.TRAINING_SET), max_parameter)
dfw_val = DataFrameWrapper(read_df_by_keys(data_files, KEYS.VALIDATION_SET), max_parameter)
dfw_test = DataFrameWrapper(read_df_by_keys(data_files, KEYS.TEST_SET), max_parameter)

In [ ]:
dfw_test_reduced = DataFrameWrapper(dfw_test.include(variable_qp=True, variable_rate=True), max_parameter)

In [ ]:
video_keys = dfw.video_keys
val_video_keys = dfw_val.video_keys
test_video_keys = dfw_test.video_keys

ax = plt.gca()
plot_sa_ta(ax, dfw.df, video_keys, "Training")
plot_sa_ta(ax, dfw_val.df, val_video_keys, "Validation", marker="s")
plot_sa_ta(ax, dfw_test.df, test_video_keys, "Test", marker="d")
plt.legend(loc="lower right")

tikzplotlib.save("notebooks/sata_cif.tex")

In [ ]:
in_train = dfw.get_ml_input()
in_val = dfw_val.get_ml_input()

out_train = dfw.get_ml_output()
out_val = dfw_val.get_ml_output()

In [ ]:
input_scaler = StandardScaler().fit(np.concatenate([in_train, in_val], axis=0))
output_scaler = StandardScaler().fit(np.concatenate([out_train, out_val], axis=0))

In [ ]:
model = tf.keras.models.load_model(MODEL_PATH)
model.summary()

In [ ]:
validate(model, dfw, input_scaler, output_scaler, "training")

In [ ]:
validate(model, dfw_val, input_scaler, output_scaler, "validation")

In [ ]:
validate(model, dfw_test, input_scaler, output_scaler, "test")

In [ ]:
dfw_all = pd.concat([dfw.df, dfw_test.df])
validate(model, DataFrameWrapper(dfw_all, max_parameter), input_scaler, output_scaler, "all")

In [ ]:
dfw_single = DataFrameWrapper(read_df_by_keys(data_files, [KEYS.SINGLE_VIDEO]), max_parameter)
in_test = input_scaler.transform(dfw_single.get_ml_input())
in_test = tf.convert_to_tensor(in_test, dtype=tf.float32)

# Measure prediction time
start = time.time()
model.predict(in_test)
end = time.time()
print(f"Average runtime ML model: {(end - start) / len(in_test)}s")

output = model.predict(in_test)
out_prediction = output_scaler.inverse_transform(output)

In [ ]:
bitrate_measured = dfw_single.get_ml_output()
plot_bitrates(np.transpose(bitrate_measured) * 0.001, np.transpose(out_prediction) * 0.001, KEYS.SINGLE_VIDEO)
save_plot(KEYS.SINGLE_VIDEO, Path("notebooks"))